In [1]:
! pip install pandas 
! pip install requests

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------------------- ------------------ 30.7/61.0 kB 660.6 kB/s eta 0:00:01
     ---------------------------------------- 61.0/61.0 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   -- ------------------------------------- 0.6/11.6 MB 12.4 MB/s eta 0:00:01
   --------------- ------------------------ 4.5/11.6 MB 48.0 MB/s eta 0:00:01
   ----------------------------------- ---- 10.2/11.6 MB 72.3 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 81.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------- ----------------------------- 4.3/15.8 MB 139.7 MB/s eta 0:00:01
   ------------- -------------------------- 5.2/15.8 MB 111.3 MB/s eta 0:00:01
   ------------- -------------------------- 5.2/15.8 MB 111.3 MB/s eta 0:00:01
   ------------- -------------------------- 5.2/15.8 MB 111.3 MB/s eta 0:00:0

In [17]:
import requests
import csv
import re

def extract_skills(description):
    """Extrae habilidades de la descripción utilizando expresiones regulares."""
    skills = re.findall(r'\b[A-Za-z0-9\-\.]+\b', description)
    return ', '.join(skills)

# Obtener todas las categorías
categories_url = "https://www.getonbrd.com/api/v0/categories?per_page=100&page=1"
categories_response = requests.get(categories_url)

if categories_response.status_code == 200:
    categories_data = categories_response.json()
else:
    print(f"Error: {categories_response.status_code}")
    categories_data = None

if categories_data:
    all_jobs = []
    for category in categories_data['data']:
        category_id = category['id']
        page = 1
        while True:
            jobs_url = f"https://www.getonbrd.com/api/v0/categories/{category_id}/jobs?per_page=100&page={page}&expand=[%22company%22]"
            jobs_response = requests.get(jobs_url)
            
            if jobs_response.status_code == 200:
                jobs_data = jobs_response.json()
            else:
                print(f"Error: {jobs_response.status_code}")
                jobs_data = None

            if jobs_data and "data" in jobs_data and jobs_data["data"]:
                for job in jobs_data["data"]:
                    attributes = job['attributes']
                    company = attributes['company']['data']['attributes'] if 'company' in attributes else {}

                    description = attributes.get('description', '')
                    skills = extract_skills(description)

                    response_time_min = attributes.get('response_time_in_days', {}).get('min', None)
                    response_time_max = attributes.get('response_time_in_days', {}).get('max', None)

                    filtered_job = {
                        "category": attributes.get('category_name', ''),
                        "seniority": attributes['seniority']['data']['type'] if 'seniority' in attributes else '',
                        "min_salary": attributes.get('min_salary', ''),
                        "max_salary": attributes.get('max_salary', ''),
                        "id": job['id'],
                        "modality": attributes['modality']['data']['type'] if 'modality' in attributes else '',
                        "remote": attributes.get('remote', ''),
                        "remote_modality": attributes.get('remote_modality', ''),
                        "remote_zone": attributes.get('remote_zone', ''),
                        "published_at": attributes.get('published_at', ''),
                        "application_deadline": attributes.get('application_deadline', ''),
                        "experience": attributes.get('experience', ''),
                        "skills": skills,
                        "benefits": attributes.get('benefits', ''),
                        "perks": ', '.join(attributes.get('perks', [])),
                        "applications_count": attributes.get('applications_count', ''),
                        "response_time_min": response_time_min,
                        "response_time_max": response_time_max,
                        "company_name": company.get('name', ''),
                        "company_country": company.get('country', ''),
                        "company_id": company.get('id', ''),
                        "company_description": company.get('description', '')
                    }
                    all_jobs.append(filtered_job)
                page += 1
            else:
                break

    # Guardar trabajos en un archivo CSV
    with open('../data/all_jobs_data.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Escribir los encabezados
        writer.writerow([
            "category", "seniority", "min_salary", "max_salary", "id", "modality", "remote", 
            "remote_modality", "remote_zone", "published_at", "application_deadline", 
            "experience", "skills", "benefits", "perks", "applications_count", "response_time_min", 
            "response_time_max", "company_name", "company_country", "company_id", "company_description"
        ])

        for job in all_jobs:
            writer.writerow([
                job["category"],
                job["seniority"],
                job["min_salary"],
                job["max_salary"],
                job["id"],
                job["modality"],
                job["remote"],
                job["remote_modality"],
                job["remote_zone"],
                job["published_at"],
                job["application_deadline"],
                job["experience"],
                job["skills"],
                job["benefits"],
                job["perks"],
                job["applications_count"],
                job["response_time_min"],
                job["response_time_max"],
                job["company_name"],
                job["company_country"],
                job["company_id"],
                job["company_description"]
            ])

    print("Todos los trabajos se han guardado en 'all_jobs_data.csv' correctamente.")
else:
    print("No se pudieron obtener las categorías.")


In [16]:
# Leer el archivo CSV con pandas para mostrar los primeros registros y verificar que se guardaron correctamente

import pandas as pd

df = pd.read_csv('../data/all_jobs_data.csv')
df.head()


,category,seniority,min_salary,max_salary,id,modality,remote,remote_modality,remote_zone,published_at,application_deadline,experience,skills,benefits,perks,applications_count,company_name,company_country,company_id,company_description
0,Programming,seniority,2400.0,4000.0,programador-biometria-aplicada-santiago,modality,False,no_remote,NaN,1716506997,NaN,NaN,"p, Debes, tener, experiencia, comprobable, en,...",<p>Somos una empresa con muy buen ambiente de ...,"health_coverage, computer_provided, informal_d...",9,Biometria Aplicada,CL,NaN,Somos una empresa sólida con más de 12 años de...
1,Programming,seniority,NaN,NaN,tech-lead-arkho-santiago-70da,modality,True,fully_remote,Santiago Metropolitan,1716436734,NaN,NaN,"div, son, los, requerimientos, del, cargo, div...",<ul><li>Día administrativo semestral hasta los...,"remote_full, flexible_hours, informal_dresscod...",190,ARKHO,CL,NaN,Somos una consultora experta en tecnologías de...
2,Programming,seniority,NaN,NaN,desarrollador-a-backend-springboot-empresas-sb...,modality,False,hybrid,NaN,1716413884,NaN,NaN,"ul, li, Experiencia, en, desarrollo, de, soluc...",NaN,"wellness, accessible, meals_provided, commutin...",42,Empresas SB,CL,NaN,Tendrás grandes desafíos y nuestro respaldo.\r...
3,Programming,seniority,NaN,NaN,desarrollador-a-ruby-on-rails-empresas-sb-sant...,modality,False,hybrid,NaN,1716229366,NaN,NaN,"ul, li, Experiencia, de, al, menos, 3, en, y, ...",NaN,"wellness, accessible, meals_provided, bicycle_...",18,Empresas SB,CL,NaN,Tendrás grandes desafíos y nuestro respaldo.\r...
4,Programming,seniority,NaN,NaN,arquitecto-java-bilingue-kibernum-bogota,modality,True,remote_local,NaN,1715899810,NaN,NaN,"p, em, Requisitos, em, p, ol, li, Profesional,...",<ul><li>Tipo de Trabajo: Remoto </li><li>Tipo ...,computer_provided,11,Kibernum,CL,NaN,Impulsamos la evolución digital de nuestro ent...
